NOTE: I'm providing a 3 implementations below with increasing complexity because if you only use the standard circuit that gives you p(0) = cos²(π·α), there’s an inherent ambiguity. This function is symmetric, meaning it can’t distinguish between α and 1 – α. The circuit “sees” only the effective phase modulo 1, and for any measurement you get two possible α values (one in [0, 0.5] and its complement in [0.5, 1]).

To resolve this ambiguity and find α uniquely, one would need to add an additional measurement that can capture the sign (or full phase) information. One approach is to perform an extra measurement—such as a tomography-like measurement that obtains the expectation values of both Pauli‑X and Pauli‑Y on the control qubit—to reconstruct the full phase unambiguously.

In [7]:
import pennylane as qml
import numpy as np

alpha = np.random.uniform(0, np.pi/2)
print("True alpha:", alpha)

dev = qml.device("default.qubit", wires=2, shots=10000)

@qml.qnode(dev)
def phase_estimation_circuit(alpha):
    qml.PauliX(wires=1)
    qml.Hadamard(wires=0)
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    qml.Hadamard(wires=0)
    return qml.sample(wires=0)

samples = phase_estimation_circuit(alpha)

p0 = np.mean(samples == 0)
print("Estimated probability of measuring 0:", p0)

alpha_est = np.arccos(np.sqrt(p0)) / np.pi
print("Estimated alpha:", alpha_est)


True alpha: 0.9138306685445374
Estimated probability of measuring 0: 0.9269
Estimated alpha: 0.08714606739490566


In [2]:
import pennylane as qml
import numpy as np

alpha = np.random.uniform(0, np.pi/2)

alpha_effective = alpha % 1
print("Effective alpha (alpha mod 1):", alpha_effective)

dev = qml.device("default.qubit", wires=2, shots=1000)

@qml.qnode(dev)
def phase_estimation_circuit(alpha):
    qml.PauliX(wires=1)
    qml.Hadamard(wires=0)
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    qml.Hadamard(wires=0)
    return qml.sample(wires=0)

samples = phase_estimation_circuit(alpha)
p0 = np.mean(samples == 0)
print("Estimated probability of measuring 0:", p0)

alpha_est = np.arccos(np.sqrt(p0)) / np.pi
alpha_alt = 1 - alpha_est

print("Estimated effective alpha from the circuit could be either:")
print("Option 1 (direct inversion):", alpha_est)
print("Option 2 (complement):", alpha_alt)


Effective alpha (alpha mod 1): 0.3785992634402485
Estimated probability of measuring 0: 0.143
Estimated effective alpha from the circuit could be either:
Option 1 (direct inversion): 0.37655918158110135
Option 2 (complement): 0.6234408184188986


In [3]:
import pennylane as qml
import numpy as np

alpha = np.random.uniform(0, np.pi/2)
print("True alpha:", alpha)

alpha_effective = alpha % 1
print("Effective alpha (alpha mod 1):", alpha_effective)

dev_standard = qml.device("default.qubit", wires=2, shots=1000)
# Tomography device to extract phase information
dev_tomo = qml.device("default.qubit", wires=2, shots=1000)

@qml.qnode(dev_standard)
def standard_circuit(alpha):
    qml.PauliX(wires=1)
    qml.Hadamard(wires=0)
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    qml.Hadamard(wires=0)
    return qml.sample(wires=0)

@qml.qnode(dev_tomo)
def tomography_circuit(alpha):
    qml.PauliX(wires=1)
    qml.Hadamard(wires=0)
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    exp_x = qml.expval(qml.PauliX(0))
    exp_y = qml.expval(qml.PauliY(0))
    return exp_x, exp_y

samples = standard_circuit(alpha)
p0 = np.mean(samples == 0)
alpha_est_standard = np.arccos(np.sqrt(p0)) / np.pi
print("\nStandard circuit:")
print("Estimated probability of measuring 0:", p0)
print("Estimated effective alpha from p(0):", alpha_est_standard)
print("Complementary possibility:", 1 - alpha_est_standard)

# Run the tomography circuit to extract the full phase
expX, expY = tomography_circuit(alpha)
phi = np.arctan2(expY, expX)
phi = phi if phi >= 0 else (phi + 2 * np.pi)
alpha_est_tomo = phi / (2 * np.pi)
print("\nTomography circuit:")
print("⟨X⟩ =", expX, " ⟨Y⟩ =", expY)
print("Extracted phase (phi):", phi)
print("Estimated effective alpha from tomography:", alpha_est_tomo)


True alpha: 1.2864672954615528
Effective alpha (alpha mod 1): 0.2864672954615528

Standard circuit:
Estimated probability of measuring 0: 0.358
Estimated effective alpha from p(0): 0.2958307862646798
Complementary possibility: 0.7041692137353202

Tomography circuit:
⟨X⟩ = -0.238  ⟨Y⟩ = 0.974
Extracted phase (phi): 1.810453317856403
Estimated effective alpha from tomography: 0.2881425947739689
